In [1]:
# =========================
# ETL Script with Oracle DB
# =========================

# Install Required Libraries (uncomment if needed)
# !pip install sqlalchemy cx_Oracle pandas schedule

# === Import Libraries ===
import cx_Oracle
import pandas as pd
import datetime
import sqlalchemy
from sqlalchemy import create_engine, text
import schedule
import time

# === Step 1: Define Database Connection ===
dsn = cx_Oracle.makedsn("localhost", 1521, sid="xe")
oracle_connection_string = f"oracle+cx_oracle://hr:hr@{dsn}"
engine = create_engine(oracle_connection_string)

# === Step 2: Function to Get Last ETL Run Timestamp ===
def get_last_etl_run(engine):
    try:
        with engine.connect() as connection:
            result = connection.execute(text("""
                SELECT MAX(last_updated) AS last_etl_run
                FROM RAW_EMPLOYEES11
            """))
            last_etl_run = result.scalar()
            if last_etl_run is None:
                print("No records found in the table.")
            else:
                print("Last ETL run timestamp:", last_etl_run)
    except Exception as e:
        print(f"An error occurred: {e}")
        last_etl_run = None

    return last_etl_run

# === Step 3: Get Last ETL Run ===
last_etl_run = get_last_etl_run(engine)

# Convert to string if datetime
if isinstance(last_etl_run, datetime.datetime):
    last_etl_run = last_etl_run.strftime('%Y-%m-%d %H:%M:%S')

# === Step 4: Query for New/Updated Records ===
emp_query = f"""
SELECT employee_id, first_name, last_name, email, hire_date, job_id, salary, department_id, raw_load_date, last_updated
FROM RAW_EMPLOYEES1 WHERE last_updated > TO_DATE('{last_etl_run}', 'YYYY-MM-DD HH24:MI:SS')
"""

query_df = pd.read_sql(emp_query, engine)
print("Data to be processed:")
print(query_df)

# === Step 5: MERGE Statement ===
merge_statement = """
MERGE INTO RAW_EMPLOYEES11 t
USING (
    SELECT 
        :employee_id AS employee_id, 
        :first_name AS first_name, 
        :last_name AS last_name, 
        :email AS email, 
        :hire_date AS hire_date, 
        :job_id AS job_id, 
        :salary AS salary, 
        :department_id AS department_id, 
        :raw_load_date AS raw_load_date, 
        :last_updated AS last_updated 
    FROM dual
) s
ON (t.employee_id = s.employee_id)
WHEN MATCHED THEN
    UPDATE SET 
        t.first_name = s.first_name, 
        t.last_name = s.last_name, 
        t.email = s.email, 
        t.hire_date = s.hire_date, 
        t.job_id = s.job_id, 
        t.salary = s.salary, 
        t.department_id = s.department_id, 
        t.raw_load_date = s.raw_load_date, 
        t.last_updated = s.last_updated
WHEN NOT MATCHED THEN
    INSERT (employee_id, first_name, last_name, email, hire_date, job_id, salary, department_id, raw_load_date, last_updated)
    VALUES (s.employee_id, s.first_name, s.last_name, s.email, s.hire_date, s.job_id, s.salary, s.department_id, s.raw_load_date, s.last_updated)
"""

# === Step 6: Execute MERGE for Each Row ===
with engine.begin() as connection:
    for index, row in query_df.iterrows():
        try:
            connection.execute(
                text(merge_statement),
                {
                    'employee_id': row['employee_id'],
                    'first_name': row['first_name'],
                    'last_name': row['last_name'],
                    'email': row['email'],
                    'hire_date': row['hire_date'],
                    'job_id': row['job_id'],
                    'salary': row['salary'],
                    'department_id': row['department_id'],
                    'raw_load_date': row['raw_load_date'],
                    'last_updated': row['last_updated']
                }
            )
            print(f"Processed employee_id: {row['employee_id']}")
        except Exception as e:
            print(f"Failed to process employee_id: {row['employee_id']} due to {e}")


  Using cached cx_Oracle-8.3.0.tar.gz (363 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build cx_Oracle


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [8 lines of output]
  <string>:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  C:\Users\Administrator\AppData\Local\Temp\pip-build-env-rdfzzwrr\overlay\Lib\site-packages\setuptools\config\expand.py:126: SetuptoolsWarning: File 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\pip-install-65weuy64\\cx-oracle_d862366f870a40bb86726dc47403983a\\README.md' cannot be found
    return '\n'.join(
  running bdist_wheel
  running build
  running build_ext
  building 'cx_Oracle' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cx_Oracle
ERROR: Could not build wheels for cx_Oracle, which is required to ins

  Using cached cx_Oracle-8.3.0.tar.gz (363 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build cx_Oracle


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [8 lines of output]
  <string>:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  C:\Users\Administrator\AppData\Local\Temp\pip-build-env-v7mgd7bz\overlay\Lib\site-packages\setuptools\config\expand.py:126: SetuptoolsWarning: File 'C:\\Users\\Administrator\\AppData\\Local\\Temp\\pip-install-2e5m25gi\\cx-oracle_000ef56f95494eba8514f2dee72fb99d\\README.md' cannot be found
    return '\n'.join(
  running bdist_wheel
  running build
  running build_ext
  building 'cx_Oracle' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cx_Oracle
ERROR: Could not build wheels for cx_Oracle, which is required to ins

ModuleNotFoundError: No module named 'cx_Oracle'